<a href="https://colab.research.google.com/github/Ahm1027/Payloads-Model/blob/master/Payload_Model_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def 

In [ ]:
def walkAndResolve(FOLDER_PATHS):
  dict = {}
  for files in os.listdir(FOLDER_PATHS):
    #read and extract hex

resolveMalwareDirs(r"/content/drive/MyDrive/Payloads")

In [ ]:
def extractHex(FOLDER_PATH):

  if len(files) != 0:
    for M_File in files:
      if str.lower(M_File[-4:]) in requiredFormats or str.lower(M_File[-4:]) in otherFormats:
        print(os.path.join(path, M_File))
        dump = get_hexdump(str(os.path.join(path, M_File)))
        if (len(FileDump) == 0):
          if dump:
            FileDump = dump
        else:
          if dump:
            FileDump.append(dump)
      elif len(os.path.splitext(M_File)) == 1:
        if (M_File[-4:] != ".rar" and M_File[-4:] != ".7z" and M_File[-4:] != ".zip"):
          dump = get_hexdump(str(os.path.join(path, M_File)))
          if (len(FileDump) == 0):
            if dump:
              FileDump = dump
          else:
            if dump:
              FileDump.append(dump)

    root_sample_name = parts[3]
    i += 1
    print(i)
    MalwaresHex[root_sample_name] = FileDump
  return MalwaresHex

hex = extractHex(r"/content/Malwares/Original")
print(hex)

In [ ]:
def get_hexdump(executable):
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  data2hex = bin2hex(data)
  output = generate_ngrams(text=data2hex, WordsToCombine=4)
  return output

def bin2hex(data):
  data2hex = ""
  for byte in data:
    hex = str(binascii.hexlify(bytes([byte]))).split("'")
    if (len(hex) >= 2):
      data2hex += hex[1]+" "
    else:
      data2hex += hex[0]+" "
  return data2hex

def generate_ngrams(text, WordsToCombine):
    words = text.split()
    output = []  
    for i in range(len(words)- WordsToCombine+1):
      word = ''
      for i in range (i, i+WordsToCombine):
        word = word + words[i]
      output.append(word)
    return output

def calculateTotalCounts(uniqueOutput, output):
  Uniquecounts = []
  for i in range(len(uniqueOutput)):
    count = 0
    for j in range(len(output)):
      if output[j] == uniqueOutput[i]:
        count = count+1
    Uniquecounts.append(count)
  return Uniquecounts

def vectorize(data):
  vectorizer = CountVectorizer()
  vectorizer.fit(data)
  vector = vectorizer.transform(data)
  return vector

In [10]:
for files in os.listdir():
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  print(data)
  break

NameError: ignored